## BERT Knowledge Representation
We want to compute, at each step of the user session (i.e. each document clicked), how their internal knowledge representation changes. Therefore, we have a few different methods to do so using Bert embeddings as a starting point.
A few assumptions:
- user starts with an empty knowledge representation
- User READS every document, and that is added to their knowledge

Document embeddings for BERT can come in a few different forms. Check `Compute_BERT_embeddings.ipynb` for how we compute each:
- SUM: sum of the embeddings for each sentence of the document
- MEAN: mean of the embeddings for each sentence of the document
- TRUNC: Truncate the document at the first 384 tokens.
- maxp_pairwise: Considering all sentences from the Wikipedia topic and the document, consider only the sentence with the higher similarity for any Wikipedia sentence
- maxp_sum: Consider only the sentence with higher similarity to the SUM of the wikipedia sentences
- maxp_mean: Consider only the sentence with higher similarity to the MEAN of the wikipedia sentences
- maxp_trunc: Consider only the sentence with higher similarity to the truncated wikipedia document
    
These are the ways we can compute the users' knowledge evolution. Will be compared to the same method of aggregation on the Wikipedia text

- MEAN: Concatenate all of the documents, the MEAN of these is the final knowledge.
- SUM: As the user clicks on documents, SUM the embeddings


In [130]:
import json
import pickle
import urllib.parse
from collections import defaultdict
from pprint import pprint

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr, kendalltau
from scipy.spatial.distance import cosine, euclidean
from sklearn.preprocessing import normalize
from tqdm.auto import tqdm
dataset = json.load(open("../data/logs_with_position.json"))

In [157]:


def load_embeddings(name, doc="docs", normalized=normalized):
    if not normalized:
        return pickle.load(open(f"../data/bert_embeddings/{base_model}_{doc}_{name}_embeddings_un.pkl", "rb"))
    else:
        return pickle.load(open(f"../data/bert_embeddings/{base_model}_{doc}_{name}_embeddings.pkl", "rb"))


dataset = json.load(open("../data/logs_with_position.json"))

embeddings = {
    "docs_mean": load_embeddings("mean"),
    "docs_sum": load_embeddings("sum"),
    "docs_trunc": load_embeddings("trunc"),
    "maxp_pairwise": load_embeddings("maxp_pairwise"),
    "maxp_sum": load_embeddings("maxp_sum"),
    "maxp_mean": load_embeddings("maxp_mean"),
    "maxp_trunc": load_embeddings("maxp_trunc"),
}

wikipedia_embeddings = {
    "sum": load_embeddings("sum", "wikipedia"),
    "mean": load_embeddings("mean", "wikipedia"),
    "trunc": load_embeddings("trunc", "wikipedia"),
}

In [171]:
normalized = True
# normalized = False
# base_model, hidden_size = "all-MiniLM-L6-v2", 384
base_model, hidden_size = "msmarco-MiniLM-L6-cos-v5", 384
# base_model, hidden_size = "all-mpnet-base-v2", 768
# f_dist = euclidean
f_dist = cosine



methods = list(embeddings.keys())
users_knowledge_MEAN = []  # add final score for the user
users_knowledge_SUM = []
final_knowledges = defaultdict(OrderedDict)
missing_docs = set()

for u in dataset:
    u_id = u["userID"]
    ALG = u["ALG"]
    RPL = u["RPL"]

    user_knowledge_mean = {k: [] for k in methods}
    user_knowledge_sum = {k: np.zeros(hidden_size) for k in methods}
    topic = urllib.parse.quote(u["topic_title"])
    clicks = 0
    for d in u["clicks"]:
        url = d["url"]
        if url not in embeddings["docs_mean"] or not np.any(embeddings["docs_mean"][url]):
            missing_docs.add(url)
            continue
        clicks += 1
        for method in methods:
            emb = embeddings[method][url]
            user_knowledge_mean[method].append(emb)
            user_knowledge_sum[method] += emb
    # normalize and compute final similarity
    for method in methods:
        if normalized:
            knowledge_mean = normalize(np.mean(user_knowledge_mean[method], axis=0).reshape(1,-1)).flatten()
            knowledge_sum = normalize(user_knowledge_sum[method].reshape(1,-1)).flatten()
        else:
            knowledge_mean = np.mean(user_knowledge_mean[method], axis=0)
            knowledge_sum = user_knowledge_sum[method]
            
        for emb_type in wikipedia_embeddings.keys():
            wiki_emb = wikipedia_embeddings[emb_type][topic]
            sum_dict = {"RPL": RPL, "ALG": ALG, "final_sim": 1 - f_dist(knowledge_sum, wiki_emb)}
            mean_dict = {"RPL": RPL, "ALG": ALG, "final_sim": 1 - f_dist(knowledge_mean, wiki_emb)}

            final_knowledges[f"SUM_{method}_{emb_type}"][u_id] = sum_dict
            final_knowledges[f"MEAN_{method}_{emb_type}"][u_id] = mean_dict

pickle.dump(dict(final_knowledges), open(f"../data/{base_model}_knowledge_gains.pkl", "wb"))
with open("../data/missing_docs.txt", "w") as outf:
    for u in missing_docs:
        outf.write(f"{u}\n")

## Correlation between learning and embedding methods

In the end, we want to know, for each of the 42 methods tried above, which one has the higher correlation with learning gains.

In [172]:
RPLs = []
ALGs = []
u_ids = [u["userID"] for u in dataset]

# make sure we follow the same order of users
pearsons_ALG = {}
spearman_ALG = {}
kendalltau_ALG = {}
corr_ALG = {}

for m in final_knowledges:
    u_ids = final_knowledges[m].keys()
    ALGs = [final_knowledges[m][u]["ALG"] for u in u_ids]
    RPLs = [final_knowledges[m][u]["RPL"] for u in u_ids]
    results = [final_knowledges[m][u]['final_sim'] for u in u_ids]
    
    # results = [_users[x]["final_sim"] for x in u_ids if x in _users]
    pearsons_ALG[m] = pearsonr(results, RPLs)[0]
    spearman_ALG[m] = spearmanr(results, RPLs)[0]
    
    pearsons_ALG[m] = pearsonr(results, ALGs)[0]
    spearman_ALG[m] = spearmanr(results, ALGs)[0]
    

pprint(sorted(pearsons_ALG.items(), key=lambda x: x[1], reverse=True)[:3])
pprint(sorted(spearman_ALG.items(), key=lambda x: x[1], reverse=True)[:3])

[('MEAN_docs_mean_sum', 0.2593284447567473),
 ('MEAN_docs_sum_sum', 0.25932843429286573),
 ('MEAN_docs_mean_mean', 0.25932841030825954)]
[('SUM_docs_mean_sum', 0.29791497849551424),
 ('MEAN_docs_mean_sum', 0.29791497849551424),
 ('SUM_docs_mean_mean', 0.29791497849551424)]


## Cross-encoder experiments

In [96]:
#get all methods
model_name = "stsb-roberta-base"
methods = [x.split("_", maxsplit=1)[1].replace(".pkl",  "") for x in os.listdir("../data/cross_encoder_scores/")]
embeddings = {x: pickle.load(open(f"../data/cross_encoder_scores/{model_name}_{x}.pkl", "rb")) for x in methods}
final_knowledges = defaultdict(OrderedDict)

for u in dataset:
    u_id = u["userID"]
    ALG = u["ALG"]
    if ALG == 0:
        continue
    RPL = u["RPL"]

    user_knowledge_mean = {k: [] for k in methods}
    user_knowledge_sum = {k: 0.0 for k in methods}
    topic = urllib.parse.quote(u["topic_title"])
    clicks = 0
    for d in u["clicks"]:
        url = d["url"]
        if url not in embeddings[methods[0]] or not np.any(embeddings[methods[0]][url]):
            continue
        for method in methods:
            score = embeddings[method][url]
            user_knowledge_mean[method].append(score)
            user_knowledge_sum[method] += score
    for method in methods:
        if len(user_knowledge_mean[method]) == 0:
            knowledge_mean = 0.0
        else:
            knowledge_mean = np.mean(user_knowledge_mean[method])
        knowledge_sum = user_knowledge_sum[method]
        if knowledge_sum != 0:
            final_knowledges[f"SUM_{method}"][u_id] = {"RPL": RPL, "ALG": ALG, "final_sim": knowledge_sum}
        if knowledge_mean !=0:
            final_knowledges[f"MEAN_{method}"][u_id] = {"RPL": RPL, "ALG": ALG, "final_sim": knowledge_mean}


In [75]:
pickle.dump(dict(final_knowledges), open(f"../data/{model_name}_knowledge_gains.pkl", "wb"))